In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import time

In [2]:
# Create a new set of 2,000 random latitudes and longitudes.
lats = np.random.uniform(-90.000, 90.000, size=2000)
lngs = np.random.uniform(-180.000, 180.000, size=2000)
lats_lngs = zip(lats, lngs)
lats_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lats_lngs)
coordinates

[(84.73042806497259, -136.81020684200362),
 (65.54884893413299, 9.772332253082993),
 (33.15987794481799, -122.00698974376826),
 (-61.56421379057672, 83.59719301920762),
 (40.33159130101825, 115.36442739068883),
 (21.080844598112563, -174.61138217178686),
 (-4.280492778092068, 28.42789039153641),
 (-57.42772584400743, -13.759428786368147),
 (-68.60402275508132, 15.830713239836712),
 (35.85820646464009, -28.49862529165503),
 (73.28308226002969, -136.75110781149542),
 (64.88083615107945, -151.13702610084349),
 (-58.80776549014679, -32.508777006405126),
 (34.495298475721, -49.801015164722344),
 (76.68860270084366, -5.14075767728329),
 (40.16606496459613, 113.25545182628815),
 (74.71493090109925, -7.672866785210601),
 (-74.5319858061123, 90.15209216073254),
 (33.81873593667875, 0.6122665779748218),
 (13.383642085269898, -43.875605895061256),
 (-44.35911943629459, -163.13586461680507),
 (69.44444669389057, -124.760476983795),
 (50.940224190708676, 165.28768184886007),
 (-21.813579890805215, 

In [4]:
from citipy import citipy
import pandas as pd
from keys import weather_api_key
from config import g_key


In [5]:
# Get the nearest city using the citipy module.
cities = list()
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    if city not in cities:
        cities.append(city)
print(cities[:10], 'Generate', len(cities))

['tuktoyaktuk', 'rorvik', 'lompoc', 'busselton', 'xuanhua', 'kapaa', 'rumonge', 'cape town', 'bredasdorp', 'ribeira grande'] Generate 767


In [6]:
# List of city data
city_data = []
# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")
# Create counters
record_count = 1
set_count = 1
# Loop through all the cities in our list
for i, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count
    record_count += 1
    # Run an API request for each of the cities
    try:
        city_weather = requests.get(city_url).json()
        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]
            # Append the City information into city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Current Description": city_weather_description,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country})
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------


NameError: name 'url' is not defined

In [ ]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

In [ ]:
new_column_order = ["City", "Country", "Lat",
                    "Lng", "Max Temp", "Humidity",
                    "Cloudiness", "Wind Speed",
                    "Current Description"]
# Assign a new or the same DataFrame the new column order.
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

In [ ]:
#Create the output File (CSV)
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the city_data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")